In [1]:
from collections import deque
import numpy as np
import grid2op

from grid2op.Runner import Runner
from grid2op.Converter import IdToAct
from grid2op.Agent.agentWithConverter import AgentWithConverter

In [2]:
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, Dense, subtract, add
from tensorflow.keras.layers import Input, Lambda

In [3]:
env_name = "rte_case14_realistic"
env = grid2op.make(env_name)

In [4]:
class TrainingParam(object):
    """
    A class to store the training parameters of the models. It was hard coded in the notebook 3.
    """
    def __init__(self,
                 DECAY_RATE=0.9,
                 BUFFER_SIZE=40000,
                 MINIBATCH_SIZE=64,
                 TOT_FRAME=3000000,
                 EPSILON_DECAY=10000,
                 MIN_OBSERVATION=50, #5000
                 FINAL_EPSILON=1/300,  # have on average 1 random action per scenario of approx 287 time steps
                 INITIAL_EPSILON=0.1,
                 TAU=0.01,
                 ALPHA=1,
                 NUM_FRAMES=1,
    ):
        print('TrainingParam __init__')
        self.DECAY_RATE = DECAY_RATE
        self.BUFFER_SIZE = BUFFER_SIZE
        self.MINIBATCH_SIZE = MINIBATCH_SIZE
        self.TOT_FRAME = TOT_FRAME
        self.EPSILON_DECAY = EPSILON_DECAY
        self.MIN_OBSERVATION = MIN_OBSERVATION   # 5000
        self.FINAL_EPSILON = FINAL_EPSILON  # have on average 1 random action per scenario of approx 287 time steps
        self.INITIAL_EPSILON = INITIAL_EPSILON
        self.TAU = TAU
        self.NUM_FRAMES = NUM_FRAMES
        self.ALPHA = ALPHA

In [5]:
class ReplayBuffer:
    """Constructs a buffer object that stores the past moves
    and samples a set of subsamples"""

    def __init__(self, buffer_size):
        self.buffer_size = buffer_size
        self.count = 0

    '''
    def add(self, s, a, r, d, s2):
        print('ReplayBuffer add')
        """Add an experience to the buffer"""
        # S represents current state, a is action,
        # r is reward, d is whether it is the end, 
        # and s2 is next state
        if np.any(~np.isfinite(s)) or np.any(~np.isfinite(s2)):
            # TODO proper handling of infinite values somewhere !!!!
            return

        experience = (s, a, r, d, s2)
        if self.count < self.buffer_size:
            self.buffer.append(experience)
            self.count += 1
        else:
            self.buffer.popleft()
            self.buffer.append(experience)

    def size(self):
        print('ReplayBuffer size')
        return self.count

    def sample(self, batch_size):
        print('ReplayBuffer sample')

        batch = []
        if self.count < batch_size:
            batch = random.sample(self.buffer, self.count)
        else:
            batch = random.sample(self.buffer, batch_size)

        # Maps each experience in batch in batches of states, actions, rewards
        # and new states
        s_batch, a_batch, r_batch, d_batch, s2_batch = list(map(np.array, list(zip(*batch))))
        return s_batch, a_batch, r_batch, d_batch, s2_batch

    def clear(self):
        print('ReplayBuffer clear')
        self.buffer.clear()
        self.count = 0
    '''

In [6]:
class RLQvalue(object):
    """
    This class aims at representing the Q value (or more in case of SAC) parametrization by
    a neural network.

    It is composed of 2 different networks:
    - model: which is the main model
    - target_model: which has the same architecture and same initial weights as "model" but is updated less frequently
      to stabilize training

    It has basic methods to make predictions, to train the model, and train the target model.
    """
    def __init__(self, action_size, observation_size,
                 learning_rate=1e-5,
                 training_param=TrainingParam()):
        # TODO add more flexibilities when building the deep Q networks, with a "NNParam" for example.
        self.action_size = action_size
        self.observation_size = observation_size
        self.learning_rate_ = learning_rate
        self.qvalue_evolution = np.zeros((0,))
        self.training_param = training_param

        self.model = None
        self.target_model = None
    
    '''
    def construct_q_network(self):
        print('RLQvalue construct_q_network')
        raise NotImplementedError("Not implemented")
    '''

    def predict_movement(self, data, epsilon):
        """Predict movement of game controler where is epsilon
        probability randomly move."""
        rand_val = np.random.random(data.shape[0])
        print(f'>> rand_val = {rand_val}')
        q_actions = self.model.predict(data)
        print(f'>> q_actions = {q_actions}')
        opt_policy = np.argmax(np.abs(q_actions), axis=-1)
        print(f'>> argmax = {opt_policy}')
        opt_policy[rand_val < epsilon] = np.random.randint(0, self.action_size, size=(np.sum(rand_val < epsilon)))
        
        self.qvalue_evolution = np.concatenate((self.qvalue_evolution, q_actions[0, opt_policy]))
        print(f'>> qvalue_evolution = {self.qvalue_evolution}')
        #print(f'>> opt_policy = {opt_policy}')
        #print(f'>> q_actions = {q_actions[0, opt_policy]}')
        return opt_policy, q_actions[0, opt_policy]
    
    '''
    def train(self, s_batch, a_batch, r_batch, d_batch, s2_batch, observation_num):
        print('RLQvalue train')
        """Trains network to fit given parameters"""
        targets = self.model.predict(s_batch)
        fut_action = self.target_model.predict(s2_batch)
        targets[:, a_batch] = r_batch
        targets[d_batch, a_batch[d_batch]] += self.training_param.DECAY_RATE * np.max(fut_action[d_batch], axis=-1)

        loss = self.model.train_on_batch(s_batch, targets)
        # Print the loss every 100 iterations.
        if observation_num % 100 == 0:
            print("We had a loss equal to ", loss)
        return np.all(np.isfinite(loss))

    @staticmethod
    def _get_path_model(path, name=None):
        print('RLQvalue _get_path_model')
        if name is None:
            path_model = path
        else:
            path_model = os.path.join(path, name)
        path_target_model = "{}_target".format(path_model)
        return path_model, path_target_model

    def save_network(self, path, name=None, ext="h5"):
        print('RLQvalue save_network')
        # Saves model at specified path as h5 file
        # nothing has changed
        path_model, path_target_model = self._get_path_model(path, name)
        self.model.save('{}.{}'.format(path_model, ext))
        self.target_model.save('{}.{}'.format(path_target_model, ext))
        print("Successfully saved network.")

    
    def load_network(self, path, name=None, ext="h5"):
        print('RLQvalue load_network')
        # nothing has changed
        path_model, path_target_model = self._get_path_model(path, name)
        self.model = load_model('{}.{}'.format(path_model, ext))
        self.target_model = load_model('{}.{}'.format(path_target_model, ext))
        print("Succesfully loaded network.")

    def target_train(self):
        print('RLQvalue target_train')
        # nothing has changed from the original implementation
        model_weights = self.model.get_weights()
        target_model_weights = self.target_model.get_weights()
        for i in range(len(model_weights)):
            target_model_weights[i] = self.training_param.TAU * model_weights[i] + (1 - self.training_param.TAU) * \
                                      target_model_weights[i]
        self.target_model.set_weights(target_model_weights)
    '''

TrainingParam __init__


In [7]:
class DuelQ(RLQvalue):
    """Constructs the desired duelling deep q learning network"""
    def __init__(self, action_size, observation_size,
                 learning_rate=0.00001,
                 training_param=TrainingParam()):
        ## print('DuelQ __init__')
        RLQvalue.__init__(self, action_size, observation_size, learning_rate, training_param)
        self.construct_q_network()

    def construct_q_network(self):
        # Uses the network architecture found in DeepMind paper
        # The inputs and outputs size have changed, as well as replacing the convolution by dense layers.
        self.model = Sequential()
        
        input_layer = Input(shape=(self.observation_size*self.training_param.NUM_FRAMES,))
        
        lay1 = Dense(self.observation_size*self.training_param.NUM_FRAMES)(input_layer)
        lay1 = Activation('relu')(lay1)
        
        lay2 = Dense(self.observation_size)(lay1)
        lay2 = Activation('relu')(lay2)
        
        lay3 = Dense(2*self.action_size)(lay2)
        lay3 = Activation('relu')(lay3)
        
        fc1 = Dense(self.action_size)(lay3)
        advantage = Dense(self.action_size)(fc1)
        fc2 = Dense(self.action_size)(lay3)
        value = Dense(1)(fc2)
        
        meaner = Lambda(lambda x: K.mean(x, axis=1) )
        mn_ = meaner(advantage)
        tmp = subtract([advantage, mn_])
        policy = add([tmp, value])

        self.model = Model(inputs=[input_layer], outputs=[policy])
        self.model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate_))

        self.target_model = Model(inputs=[input_layer], outputs=[policy])
        self.target_model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate_))
        print("Successfully constructed networks.")

TrainingParam __init__


In [8]:
class MyDeepQAgent(AgentWithConverter):
    
    ## 1*0^-5 = 0.00001
    def __init__(self, action_space, mode="DDQN", learning_rate=1e-5, training_param=TrainingParam()):     
        
        ## Handle only vectors, and the type of action_space is GridObjects
        AgentWithConverter.__init__(self, action_space, action_space_converter=IdToAct)

        # and now back to the origin implementation
        self.replay_buffer = ReplayBuffer(training_param.BUFFER_SIZE)

        # compare to original implementation, i don't know the observation space size.
        # Because it depends on the component of the observation we want to look at. So these neural network will
        # be initialized the first time an observation is observe.
        self.deep_q = None
        self.mode = mode
        self.learning_rate = learning_rate
        self.training_param = training_param
    
    def convert_obs(self, observation):
        ## print(f'>> convert_obs = {np.concatenate((observation.rho, observation.line_status, observation.topo_vect))}')
        return np.concatenate((observation.rho, observation.line_status, observation.topo_vect))

    def my_act(self, transformed_observation, reward, done=False):
        ## print(f'>> transformed_observation = {transformed_observation}')
        if self.deep_q is None:
            self.init_deep_q(transformed_observation)
        
        predict_movement_int, *_ = self.deep_q.predict_movement(transformed_observation.reshape(1, -1), epsilon=0.0)
        print(f'>> predict_movement_int = {predict_movement_int}')
        print(*_)
        return int(predict_movement_int)

    def init_deep_q(self, transformed_observation):
        if self.deep_q is None:
            # the first time an observation is observed, I set up the neural network with the proper dimensions.
            if self.mode == "DQN":
                cls = DeepQ
            elif self.mode == "DDQN":
                cls = DuelQ
            elif self.mode == "SAC":
                cls = SAC
            else:
                raise RuntimeError("Unknown neural network named \"{}\". Supported types are \"DQN\", \"DDQN\" and "
                                   "\"SAC\"".format(self.mode))
            self.deep_q = cls(self.action_space.size(), observation_size=transformed_observation.shape[-1], learning_rate=self.learning_rate)
            print(f'>> action_size = {self.deep_q.action_size}, observation_size = {self.deep_q.observation_size}, learning_rate_ = {self.deep_q.learning_rate_}, qvalue_evolution = {self.deep_q.qvalue_evolution}, training_param = {self.deep_q.training_param}, model = {self.deep_q.model}, target_model = {self.deep_q.target_model}')
            
    '''
    def load_network(self, path):
        print('MyDeepQAgent load_network')
        # not modified compare to original implementation
        self.deep_q.load_network(path)
    '''

TrainingParam __init__


In [9]:
my_agent = MyDeepQAgent(env.action_space)

runner = Runner(**env.get_params_for_runner(), agentClass=MyDeepQAgent)
res = runner.run(nb_episode=1, max_iter=10)

Successfully constructed networks.
>> action_size = 451, observation_size = 96, learning_rate_ = 1e-05, qvalue_evolution = [], training_param = <__main__.TrainingParam object at 0x7feec9ae79d0>, model = <keras.engine.functional.Functional object at 0x7feec9a7fe80>, target_model = <keras.engine.functional.Functional object at 0x7feec963b610>
>> rand_val = [0.50701507]
1/1 [==============================] - 0s 81ms/step


2022-06-02 21:00:56.082991: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


>> q_actions = [[ 1.0334537   0.53934324  0.23217481  0.425765    0.97880244  0.40077382
   0.5049721   0.39055574  0.38654286  0.41548347  0.85216475  0.59191334
   0.42569402  0.3437204   0.44283527  0.35330802  0.5534356   0.4845095
   0.22676344  0.6526748   0.59936655  0.4174549   0.42644998  0.37036374
   0.57697076  0.4471568   0.8464261   0.45388442  0.67742354 -0.06016797
   0.41905218  0.18587962  0.2699697   0.32057947  0.31662163  0.39133847
   0.5227047   0.4634252   0.50978214  0.18868291  0.44343215  0.54863477
   0.7011525   0.6938271   0.5504229   0.10351646  0.11946505  0.3524045
   0.61729     0.88002706  0.8204594   0.0349828   0.5714311   0.47080973
   0.5145987   0.32574853  0.36516696  0.5218014   0.9612078   0.4437476
   0.3981024   0.5619908   0.23989211  0.3719298   0.37932605  0.3337577
   0.6225888   0.26147625  0.65464616  0.57242984  0.7092445   0.9077025
   0.2774457   0.28036773  0.37359765  0.43656918  0.2888719   0.41803193
   0.39361766  0.50345767  0

1/1 [==============================] - 0s 12ms/step
>> q_actions = [[ 1.0343856   0.53983784  0.23283307  0.42580104  0.979175    0.40030083
   0.5050656   0.39094937  0.38677245  0.415687    0.85322964  0.5925084
   0.42607942  0.34429392  0.44271842  0.354137    0.5538438   0.48477563
   0.2278692   0.65265226  0.5993315   0.41772884  0.42665398  0.37020487
   0.5773311   0.4477593   0.84639776  0.45454326  0.6785457  -0.0595355
   0.41991535  0.18668953  0.27021378  0.3219762   0.3168522   0.39210573
   0.5233918   0.46388173  0.5107117   0.18920058  0.4436778   0.54913425
   0.70145744  0.6946087   0.55034524  0.10379753  0.12006745  0.35290405
   0.6169507   0.88034856  0.820996    0.03494757  0.57216525  0.47149512
   0.5148511   0.3264911   0.36550397  0.522081    0.96127     0.4441097
   0.39831454  0.56111276  0.2407978   0.3727237   0.38005725  0.33365577
   0.6228098   0.26213217  0.6547288   0.5731628   0.70984566  0.9076718
   0.27838504  0.28079796  0.37326634  0.43707103

>> rand_val = [0.06319756]
1/1 [==============================] - 0s 12ms/step
>> q_actions = [[ 1.0332      0.54032195  0.23438618  0.42613703  0.97991043  0.40003562
   0.5050729   0.39083096  0.38786507  0.41734686  0.85315526  0.59304535
   0.4260107   0.3445149   0.44320425  0.35527328  0.5544902   0.48554564
   0.22823966  0.652991    0.59908587  0.41835546  0.42688492  0.37044182
   0.57729447  0.44749448  0.845793    0.45533425  0.67874646 -0.05841815
   0.42060757  0.1871112   0.27083516  0.3218955   0.31781727  0.3921726
   0.5243617   0.46358478  0.51115286  0.19014454  0.4432906   0.54872763
   0.7010337   0.69432616  0.550404    0.10349321  0.1202262   0.35235092
   0.6170762   0.88084567  0.82100344  0.03501624  0.57265097  0.4718973
   0.5156196   0.32715017  0.36640725  0.5230271   0.9603107   0.44408134
   0.39804938  0.5607373   0.24127387  0.37349308  0.37935993  0.33354676
   0.622687    0.2626596   0.654096    0.5735184   0.7097076   0.9068835
   0.2790861   0.2817

>> rand_val = [0.27211775]
1/1 [==============================] - 0s 12ms/step
>> q_actions = [[ 1.0314754   0.5386282   0.23242399  0.425113    0.9780458   0.39939794
   0.50390863  0.3898308   0.38629594  0.41679204  0.85097605  0.59157544
   0.42543733  0.342958    0.44182804  0.35310853  0.55388844  0.48384467
   0.22621597  0.65229505  0.59873194  0.41704983  0.4262228   0.369833
   0.57603115  0.44660386  0.844701    0.45339364  0.67675734 -0.06003174
   0.4191775   0.18629321  0.26932853  0.32031888  0.3168208   0.3912407
   0.5220419   0.46303174  0.50936973  0.18803161  0.44167042  0.5471765
   0.69941306  0.6933905   0.5501502   0.10254386  0.11896378  0.35167873
   0.6162703   0.88013375  0.8195696   0.0343602   0.5703634   0.470136
   0.51412314  0.32500184  0.36521524  0.52171314  0.9601171   0.44363362
   0.39751834  0.56071293  0.23976988  0.37204957  0.37848338  0.3331963
   0.6217066   0.2614528   0.65289235  0.5723607   0.70829016  0.90547
   0.27778763  0.27981895  0

>> rand_val = [0.43916573]
1/1 [==============================] - 0s 14ms/step
>> q_actions = [[ 1.0270212   0.5371916   0.23286505  0.42401072  0.9768768   0.39637554
   0.50209606  0.38883036  0.38633496  0.4174232   0.8487364   0.5906585
   0.42396858  0.34089732  0.44123515  0.35275793  0.55274564  0.4822075
   0.22453284  0.651526    0.597199    0.41594428  0.42432827  0.36862323
   0.5736899   0.44528905  0.84087837  0.450906    0.6749896  -0.06054831
   0.41877335  0.18697101  0.2679981   0.31844187  0.3157667   0.3905592
   0.51977396  0.4605315   0.5076014   0.18679696  0.4383633   0.5446011
   0.695724    0.6908462   0.5481628   0.10036287  0.11734435  0.3497985
   0.6140709   0.8784783   0.8166759   0.03317761  0.5676746   0.46845922
   0.51232946  0.32433057  0.36473203  0.5211562   0.95726305  0.44223848
   0.39511847  0.5586991   0.23771888  0.37087202  0.37650567  0.33174798
   0.6192596   0.26015502  0.6500063   0.5721457   0.70525354  0.9019056
   0.27679512  0.2793281

In [10]:
for _, chron_name, cum_reward, nb_time_step, max_ts in res:
    msg_tmp = "\tFor chronics with id {}\n".format(chron_name)
    msg_tmp += "\t\t - cumulative reward: {:.6f}\n".format(cum_reward)
    msg_tmp += "\t\t - number of time steps completed: {:.0f} / {:.0f}".format(nb_time_step, max_ts)
    print(msg_tmp)

	For chronics with id 000
		 - cumulative reward: 923.063965
		 - number of time steps completed: 10 / 10
